# Quantum Circuit Compiler with Optimizer

This is a quantum circuit compiler and optimizer which converts any given Cirq circuit made up of the basic gates: I, S, H, X, Y, Z, RX, RY, RZ, CNOT, CZ into a combination of our fundamental gates: RX, RZ, and CZ. It also optimizes if those combinations result in a net zero change in the state of qubit or if the application of multiple gates can be merged to a single gate, for example: `--RX(pi/2)-RX(pi/2)--` is effectively `--RX(pi)--`.

Some of the supported features:
* Compiler supports inclusion of more gates available in Cirq as long as they can be converted to a combination of the basic gates
* Optimizer supports addition of custom optimizers and more quantum circuit identities
* Optimizer supports replacement of a list of operations with another list of operations on a single qubit
* Importing circuits from JSON files and URLs is also supported

Information about how circuits and gates work in Cirq can be found [here](https://cirq.readthedocs.io/en/stable/).

Even though this implementation is more function oriented than object oriented (because I feel like it allows for more customizations), it can be written in an object oriented manner whenever deemed appropriate.

Note: I decided to perform this on a Jupyter Notebook as it will be easier to explain the steps than writing everything as comments in files. Also, this notebook can easily be converted to a python file but not the other way round, not without losing important information at least.

Whew! Here we go.

## Importing required libraries

Importing `cirq` as we will be using its various methods to create our compiler and optimizer.

Importing `pi` from `math` as we are working with angles here.

In [1]:
import cirq
from math import pi

## Importing/Creating a Circuit

### Creating a cicruit

We will be creating a list of `cirq.LineQubit()` qubits to apply the circuit upon and `N_QUBITS` is the total number of qubits that will be required by our sample circuit.

In [2]:
# Creating qubits for the sample circuit
N_QUBITS = 5
qubits = [cirq.LineQubit(i) for i in range(N_QUBITS)]

Creating a very basic circuit with Hadamard, RX, and CNOT gates.

In [3]:
# Creat a new circuit
input_circuit = cirq.Circuit()

# Apply Hadamard Gate on first 3 qubits
input_circuit.append([cirq.H(qubits[0])])
input_circuit.append([cirq.H(qubits[1])])
input_circuit.append([cirq.H(qubits[2])])

# Apply CNOT Gate on consecutive qubits
for i in range(N_QUBITS - 1):
    input_circuit.append([cirq.CNOT(qubits[i], qubits[i + 1])])

# Apply RX(pi/2) Gate on all qubits
rx = cirq.rx(pi / 2)
for q in qubits:
    input_circuit.append([rx(q)], cirq.InsertStrategy.INLINE)                           

# Print the circuit
input_circuit

0: ───H───@───────────Rx(0.5π)──────────────
          │
1: ───H───X───@───────Rx(0.5π)──────────────
              │
2: ───H───────X───@───Rx(0.5π)──────────────
                  │
3: ───────────────X───@──────────Rx(0.5π)───
                      │
4: ───────────────────X──────────Rx(0.5π)───

Now, in Cirq circuits are executed on a "moment" by "moment" basis where a `moment` is a time slice. All operations sharing a `moment` will be executed at once. So the first `moment`, in our sample circuit above, consists of a Hadamard gate operation on qubits `qubit[0], qubit[1], qubit[2]`. After executing that, the next `moment` made of a CNOT gate operation on `qubit[0]` as control qubit and `qubit[1]` as target qubit will be executed and so on.

### Importing a Circuit

To import a circuit simply uncomment the appropriate line from the codeblock below, and provide the JSON or a url to the function.

In [4]:
# Import from JSON:
# input_circuit = cirq.quirk_json_to_circuit("path/to/json/here")

# Import from a URL:
# input_circuit = cirq.quirk_url_to_circuit("url/here")

## Compiling the Circuit
Codeblock below contains the compiler functions for each supported gate. One can simply add a new gate by implementing a new compiler function `compile_GATE()` which takes in the qubits on which to apply the gate and theta if the gate changes a phase around any axis. It should return an equivalent combination of RX, RZ and CZ gates.

For example, a CNOT Gate can be obtained by applying a CZ gate and surrounding the target qubit with Hadamard Gates. The function for that will be `compile_CNOT()` written below.

In [5]:
# Available gates for the compiled circuit
def compile_rx(qubit, theta):
    """
    --RX(theta)--
    """
    rx = cirq.rx(theta)
    return rx(qubit)


def compile_rz(qubit, theta):
    """
    --RZ(theta)--
    """
    rz = cirq.rz(theta)
    return rz(qubit)


def compile_CZ(control_q, target_q):
    """
    control_q: --@--
                 |
    target_q:  --@--
    """
    return cirq.CZ(control_q, target_q)


# Gates that were used by the input circuit apart from the ones above
def compile_ry(qubit, theta):
    """
    --RY(theta)-- is equivalent to
    --RX(pi/2)-RZ(theta)-RX(-pi/2)--
    """
    return compile_rx(qubit, pi / 2), compile_rz(qubit, theta), compile_rx(qubit, -pi / 2)


def compile_I(qubit):
    """
    --I-- is equivalent to
    --RX(2*pi)-- or
    --RZ(2*pi)-- or
    --RY(2*pi)--
    """
    return compile_rx(qubit, 2 * pi)


def compile_H(qubit):
    """
    --H-- is equivalent to
    --RZ(pi/2)-RX(pi/2)-RZ(pi/2)--
    """
    return compile_rz(qubit, pi / 2), compile_rx(qubit, pi / 2), compile_rz(qubit, pi / 2)


def compile_X(qubit):
    """
    --X-- is equivalent to
    --RX(pi)--
    """
    return compile_rx(qubit, pi)


def compile_Y(qubit):
    """
    --Y-- is equivalent to
    --RY(pi)--
    """
    return compile_ry(qubit, pi)


def compile_Z(qubit):
    """
    --Z-- is equivalent to
    --RZ(pi)--
    """
    return compile_rz(qubit, pi)


def compile_S(qubit):
    """
    --S-- is equivalent to
    --RZ(pi/2)--
    """
    return compile_rz(qubit, pi / 2)


def compile_CNOT(control_q, target_q):
    """
    control_q: --@--
                 |
    target_q:  --X--
    
    is equivalent to

    control_q: ----@----
                   |
    target_q:  --H-@-H--
    """
    return compile_H(target_q), compile_CZ(control_q, target_q), compile_H(target_q)

The method `get_compiled_ops()`, defined below, detects the gate operation being performed in a moment and calls the respective `compile_GATE()` function. It then returns the list of compiled operations. Explanation is written in the function's description.

More gates can be added by adding an `elif` statement here and passing the required parameters to the new `compile_GATE()` function accordingly.

Attributes of a gate operation `op` can be accessed as:

* `op.gate` returns the gate that was applied to the qubit(s) by the operation `op`.

* `op.qubits` returns the list of qubits upon which the operation `op` was performed. 

For more information on `cirq.GateOperation` objects click [here](https://cirq.readthedocs.io/en/stable/generated/cirq.GateOperation.html).

In [6]:
def get_compiled_ops(moment):
    """
    input: moment: cirq.Moment object
    return: compiled_operations: list of cirq.GateOperation objects

    Replaces the operations in given moment to
    a combination of fundamental gates:
    RX, RZ, and CZ 
    """
    compiled_operations = []
    
    # For each operation in a moment
    for op in moment.operations:

        # Retrieve which gate operation is being performed
        gate = op.gate

        # Compile gates accordingly:
        # Single qubit gates
        if gate == cirq.I:
            new_op = compile_I(op.qubits[0])

        elif gate == cirq.H:
            new_op = compile_H(op.qubits[0])

        elif gate == cirq.X:
            new_op = compile_X(op.qubits[0])

        elif gate == cirq.Y:
            new_op = compile_Y(op.qubits[0])

        elif gate == cirq.Z:
            new_op = compile_Z(op.qubits[0])
        
        elif gate == cirq.S:
            new_op = compile_S(op.qubits[0])
        
        # Two qubit gates
        elif gate == cirq.CNOT:
            new_op = compile_CNOT(op.qubits[0], op.qubits[1])

        elif gate == cirq.CZ:
            new_op = compile_CZ(op.qubits[0], op.qubits[1])
        
        # Single qubit rotation gates around x, y, z axis
        # on Bloch sphere with rotation by theta radians
        else:

            # Retrieve the value of theta
            theta = gate.exponent * pi

            if gate == cirq.rx(theta):
                new_op = compile_rx(op.qubits[0], theta)

            elif gate == cirq.ry(theta):
                new_op = compile_ry(op.qubits[0], theta)

            elif gate == cirq.rz(theta):
                new_op = compile_rz(op.qubits[0], theta)
            
            else:
                # If an unrecognized gate found then leave as is
                new_op = op


        # Add the new operation to the list of compiled operations
        compiled_operations.append(new_op)
        
    return compiled_operations

Compiling the circuit on a moment by moment basis.

In [7]:
# Create a new circuit to store the compiled circuit
compiled_circuit = cirq.Circuit()

# For each moment in input_circuit get compiled operations
for moment in input_circuit:
    compiled_circuit.append(get_compiled_ops(moment))

# Print the compiled circuit
compiled_circuit

0: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)────────────────────────────────────@───Rx(0.5π)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                        │
1: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)──────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                             │
2: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)─────────────────────────────────────────────────────────────────────────
                                                                                                                                                  │
3: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)──────────────────────────────────────────────────────────────────────────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)────────────────────────────────────
                                                                                                                                                                                       │
4: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───Rx(0.5π)───

In addition to the decreased visual appeal, our compiled circuit has a lot more moments than our input circuit. This increase in the number of moments can be considered as the overhead generated by our compiler. So, let's see how we can decrease this number to restrict its impact on performance when executing the circuit.

In [8]:
print("Number of moments in the input circuit: ", len(input_circuit))
print("Number of moments in the compiled output circuit: ", len(compiled_circuit))

Number of moments in the input circuit:  6
Number of moments in the compiled output circuit:  23


## Optimizing the Compiled Circuit

Below are lists of operations which if applied in series to a qubit, result in either effectively doing nothing or another single qubit operation. Thus, that list of operations can be replaced with a single operation.

For example, if our optimizer encountered similar gate operations occurring consecutively like `--RX(pi)--RX(pi/2)--`, then it will merge those two into a single gate operation `--RX(3*pi/2)--`, using `merge_similar_ops()` function defined below.

Also, it is to be noted that the operations in the lists below are written in reverse order of their application wherever it matters, since the method that compares them to the operations in the circuit, uses a stack. So, for example, if the optimizer encountered the compiled equivalent of `--Y-Y--`:


* `--RX(pi/2)--RZ(pi)--RX(-pi/2)--RX(pi/2)--RZ(pi)--RX(-pi/2)--`


which after merging similar gates, becomes:

* `--RX(pi/2)--RZ(pi)--RX(0)--RZ(pi)--RX(-pi/2)--`

but when these are put in a stack, they will become:

* `--RX(-pi/2)--RZ(pi)--RX(0)--RZ(pi)--RX(pi/2)--`

So, if we compare this stack to the list of operations `[cirq.rx(-pi / 2), cirq.rz(pi), cirq.rx(0), cirq.rz(pi), cirq.rx(pi / 2)]` (which is compiled equivalent of `--Y-Y--` but reversed) it will then return `True`. This merging is performed by the function `merge_different_ops()` defined below.


Since there are numerous quantum identities which allow replacement of multiple operations with a different set of operations, I have included the option to add more identities by simply adding a reverse ordered list of operations in the respective `can_be_merged_to_` list, or to the `can_be_merged_to_custom` list for customized replacement. Make sure to implement the corresponding `elif` condition in `merge_different_ops()` function.

Some identities that have already been implemented (apart from the ones obtained by the definition of a unitary operation):

* `--X-Y-Z--` is equivalent to `--I--`

* `--H-Z-H--` is equivalent to `--X--`

* `--H-X-H--` is equivalent to `--Z--`

In [9]:
# Gate combinations that can be merged to I, X, Z. In reversed order for comparison to a stack

can_be_merged_to_i = [

    [cirq.rx(2 * pi)],                                                                 # equivalent to --X-X--
    [cirq.rx(-pi / 2), cirq.rz(pi), cirq.rx(0), cirq.rz(pi), cirq.rx(pi / 2)],         # equivalent to --Y-Y--
    [cirq.rz(2 * pi)],                                                                 # equivalent to --Z-Z--
    [cirq.rz(pi), cirq.rx(-pi / 2), cirq.rz(pi), cirq.rx(3 * pi / 2)],                 # equivalent to reversed --X-Y-Z--
    [cirq.rz(pi / 2), cirq.rx(pi / 2), cirq.rz(pi), cirq.rx(pi / 2), cirq.rz(pi / 2)], # equivalent to --H-H--

]

can_be_merged_to_x = [
    
    [cirq.rz(pi / 2), cirq.rx(pi), cirq.rz(pi / 2)]                                    # equivalent to --H-Z-H--

]

can_be_merged_to_z = [

    [cirq.rz(pi / 2), cirq.rx(pi / 2), cirq.rz(pi / 2), cirq.rx(pi), 
     cirq.rz(pi / 2), cirq.rx(pi / 2), cirq.rz(pi / 2)]                                # equivalent to --H-X-H--
     
]

# More single qubit quantum circuit identities can be added here
can_be_merged_to_custom = [

]

### Optimizer Definitions

Descriptions about a particular optimizer can be found in their respective definitions. Note that currently the `merge_different_ops()` function merges multiple operations to a single operation but it can easily be expanded to replace a list of operations with a different list of operations by simply adding a relevant `elif` comparison statement. I didn't implement that because I couldn't find a relatively simple quantum circuit identity.

A custom optimizer can also be created by implementing the `custom_optimizer()` function.

In [10]:

def merge_similar_ops(input_ops):
    """
    input: input_ops: list of cirq.GateOperation objects
    return: opt_ops: list of cirq.GateOperation objects

    Attempts to combine consecutive rotation 
    operations which operate around the same axis and
    return an equivalent single rotation operation.

    For example,
    
    --RX(pi)-RX(pi)--
    
    will be combined to form
    
    --RX(2*pi)--
    """

    opt_ops = []                                            # List of optimized operations
    q = input_ops[0].qubits[0]                              # Retrieving the qubit on which the operations are performed
    opt_ops.append(input_ops[0])                            # Appending the first operation to "opt_ops"

    for operation in input_ops[1:]:                         # For each operation in the "input_ops" list after the first one

        if type(opt_ops[-1].gate) == type(operation.gate):      # Check whether the last gate in "opt_ops" is similar
                                                                # to the current operation's gate

            theta_1 = opt_ops[-1].gate.exponent * pi                # Retrieve theta from last gate in "opt_ops"
            theta_2 = operation.gate.exponent * pi                  # Retrieve theta from current operation's gate
            
            if opt_ops[-1].gate == cirq.rx(theta_1):                # Check if its an RX gate
                opt_gate = cirq.rx(theta_1 + theta_2)                   # Merge the two gates to create an equivalent RX gate
                opt_ops[-1] = opt_gate(q)                               # Apply the newly created RX gate to qubit "q"
            
            elif opt_ops[-1].gate == cirq.rz(theta_1):              # Check if its an RZ gate
                opt_gate = cirq.rz(theta_1 + theta_2)                   # Merge the two gates to create an equivalent RZ gate
                opt_ops[-1] = opt_gate(q)                               # Applye the newly created RZ gate to qubit "q"

            else:                                                   # If two CZ gates were found touching this qubit simply
                opt_ops.append(operation)                           # append current operation to "opt_ops" and move to next operation
        else:
            opt_ops.append(operation)                           # If gates are not similar then append current operation to "opt_ops"
                                                                # and move to next operation

    return opt_ops


def merge_different_ops(input_ops):
    """
    input: input_ops: list of cirq.GateOperation objects
    return: opt_ops: list of cirq.GateOperation objects

    Attempts to replace the given
    set of operations(input_ops) with a different
    but equivalent set of operations(opt_ops)
    in order to reduce their size, i.e if optimized,
    
    len(opt_ops) < len(input_ops)

    `stack_ops` appends operations onto `opt_ops`
    depending upon whether the combination of gates
    in `stack_gate` is present in any of the 
    `can_be_merged_to_` lists. If it is present
    then it will be replaced by an equivalent
    single gate operation. If it does not match
    any combination of gates, then this test
    will be performed again after adding another
    operation to `stack_ops` until all input 
    operations have been tested.

    Note: Identity gates will be removed by
          this optimizer
    """

    opt_ops = []                                        # List of optimized operations
    q = input_ops[0].qubits[0]                          # Retrieve the qubit on which the operations are performed
    stack_gates = []                                    # Stack to store gate combinations for comparison later on
    stack_ops = []                                      # Stack to store operations of those gates


    for operation in input_ops:                         # For each operation in input operations
        opt_ops.append(operation)                           # Append current operation to "opt_ops"

        while(len(opt_ops) > 0):                            # Run until "opt_ops" is empty
            stack_ops.append(opt_ops.pop())                     # Remove last operation in "opt_ops" and append it to "stack_ops"
            stack_gates.append(stack_ops[-1].gate)              # and append its gate into "stack_gates"
            

            if stack_gates in can_be_merged_to_i:               # Check whether gate combination in "stack_gates" can be merged to Identity
                stack_gates.clear()                                 # Empty the stacks as the gate combination effectively does nothing
                stack_ops.clear()

            elif stack_gates in can_be_merged_to_x:             # Check whether gate combination in "stack_gates" can be merged to X
                stack_gates.clear()                                 # First, empty the stacks
                stack_ops.clear()

                X = cirq.rx(pi)                                     # Then create X gate from RX(pi) since this optimizer works on
                                                                    # our fundamental gate types
                stack_ops.append(X(q))                              # Append operation X(q) to "stack_ops"
                stack_gates.append(X)                               # and append the created X gate onto "stack_gates"

            elif stack_gates in can_be_merged_to_z:             # Check whether gate combination in "stack_gates" can be merged to Z
                stack_gates.clear()                                 # First, empty the stacks
                stack_ops.clear()

                Z = cirq.rz(pi)                                     # Then create Z gate from RZ(pi) for the same reason as above

                stack_ops.append(Z(q))                              # Append operation Z(q) to "stack_ops"
                stack_gates.append(Z)                               # and append the created Z gate onto "stack_gates"

        stack_gates.clear()                                 # Empty "stack_gates" once comparison of gate combinations is done
                                                            # for operations until current operation
        
        while(len(stack_ops) > 0):                          # Run until "stack_ops" is empty
            opt_ops.append(stack_ops.pop())                     # Remove last operation from "stack_ops" and append it to "opt_ops"


    return opt_ops

def custom_optimizer(input_ops):
    """
    input: input_ops: list of cirq.GateOperation objects
    return: opt_ops: list of cirq.GateOperation objects

    Implement a custom optimizer which replaces here.
    By default it returns the list of operations as is.
    """
    return input_ops

The function `optimize()` optimizes given circuit according to the `optimizers` passed to it using `cirq.MergeSingleQubitGates` and `cirq.DropEmptyMoments`. For more information on how they work click [here](https://cirq.readthedocs.io/en/stable/api.html#optimization).

Note: This function currently supports optimizing single qubit gates only. Support for multi-qubit gates can be added using a variety of methods provided by Cirq [here](https://cirq.readthedocs.io/en/stable/api.html#optimization).

In [11]:
def optimize(circuit, optimizers):
    """
    input: circuit: cirq.Circuit object
           optimizers: list of optimizer functions
    
    return: opt_circuit: cirq.Circuit object

    This function will apply given optimizers
    on the given circuit and drop any empty
    moments if they are present.
    """

    # Cloning the input circuit since the optimization is
    # done in-place and will otherwise modify the input circuit
    opt_circuit = cirq.Circuit(circuit)
    
    # For each optimizer in optimizers
    for opt in optimizers:

        # Replace single qubit operations with optimized operations
        # as specified by the optimizer "opt"
        cirq.MergeSingleQubitGates(rewriter=opt).optimize_circuit(opt_circuit)
    
    # Drop empty moments, if present, from the optimized circuit
    cirq.DropEmptyMoments().optimize_circuit(opt_circuit)
    
    return opt_circuit

Optimizing our compiled circuit according to optimizers provided in the list `optimizers`.

Note: The order of elements in `optimizers` matters as the first optimizer will be applied first before moving to the next one. It is also highly recommended that `merge_similar_op` be used as the first optimizer for best results.

In [12]:
# List of optimizers to be used
optimizers = [merge_similar_ops, merge_different_ops]

# Obtain the optimized circuit
optimized_circuit = optimize(compiled_circuit, optimizers)

# Print the optimized circuit
optimized_circuit

0: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                       │
1: ────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)──────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                            │
2: ─────────────────────────────────────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)─────────────────────────────────────────────────────────────────────────
                                                                                                                 │
3: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)─────────────────────────────────────────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)────────────────────────────────────
                                                                                                                                                      │
4: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)──────────────────────────────────────────────────────────────────────────────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───Rx(0.5π)───

Now, after optimization we can see that there has been an improvement, as follows:

In [13]:
print("Number of Moments in the input circuit: ", len(input_circuit))
print("Number of Moments in the compiled output circuit: ", len(compiled_circuit))
print("Number of Moments in the optimized circuit: ", len(optimized_circuit))

Number of Moments in the input circuit:  6
Number of Moments in the compiled output circuit:  23
Number of Moments in the optimized circuit:  20


As can be seen, after basic optimizations we were able to reduce the overhead generated by our compiled circuit. The optimized circuit also looks less involved than before.

Keep in mind that not all circuits can be improved like this, for example if a circuit is already simplified by the optimizer's standards then it will not result in a better circuit.

In [14]:
print("Number of moments in the optimized circuit: ", len(optimized_circuit))
print("Number of moments in the double optimized circuit: ", len(optimize(optimized_circuit, optimizers)))

Number of moments in the optimized circuit:  20
Number of moments in the double optimized circuit:  20


## Future Work

There are some things which I believe can be implemented in future iterations to improve upon the current implementation:

* Organize the code in different files appropriately
* Optimizations for multi-qubit operations
* Add more quantum circuit identities
* Add support for more gates
* Add support for more kinds of qubits other than `cirq.LineQubit()`
* Look for more optimizations that can further reduce overhead generated by the compiler
* Compile given circuit into a combination of any given set of gates, i.e, implement `compile(input_circuit, list_of_fundamental_gates_to_be_used)`

Apart from these, feel free to open a PR to suggest more features that you think will be great!